In [1]:
from google.colab import files
files.upload()

Output hidden; open in https://colab.research.google.com to view.

In [2]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision
import torchvision.transforms as transforms

import numpy as np
import pandas as pd
from PIL import Image, ImageEnhance, ImageFilter
from tqdm import tqdm

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

device(type='cuda')

In [3]:
!mkdir -p failure-aware-cv/results/Model
!mv resnet18_cifar10_fc_only.pth failure-aware-cv/results/Model/

In [4]:
%cd /content/failure-aware-cv

/content/failure-aware-cv


In [5]:
model = torchvision.models.resnet18(pretrained=False)
model.fc = nn.Linear(model.fc.in_features, 10)

model.load_state_dict(
    torch.load("results/Model/resnet18_cifar10_fc_only.pth", map_location=device)
)

model = model.to(device)
model.eval()

for p in model.parameters():
    p.requires_grad = False

print("✅ Trained CIFAR-10 model loaded")

/usr/local/lib/python3.12/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)


✅ Trained CIFAR-10 model loaded


In [6]:
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor()
])

test_dataset = torchvision.datasets.CIFAR10(
    root="./data",
    train=False,
    download=True,
    transform=transform
)

test_loader = torch.utils.data.DataLoader(
    test_dataset,
    batch_size=64,   # GPU speedup
    shuffle=False,
    num_workers=2
)

100%|██████████| 170M/170M [00:03<00:00, 43.6MB/s]


In [7]:
def add_gaussian_noise(img, severity=0.3):
    arr = np.array(img).astype(np.float32) / 255.0
    noise = np.random.normal(0, severity, arr.shape)
    noisy = np.clip(arr + noise, 0, 1)
    return Image.fromarray((noisy * 255).astype(np.uint8))

def apply_blur(img, radius=2):
    return img.filter(ImageFilter.GaussianBlur(radius))

def low_light(img, factor=0.4):
    return ImageEnhance.Brightness(img).enhance(factor)

In [8]:
def compute_entropy(probs):
    probs = probs + 1e-12
    return -torch.sum(probs * torch.log(probs), dim=1)

**Combined**

In [14]:
# def run_inference(loader, degradation=None):
#     results = []

#     for images, labels in tqdm(loader):
#         for i in range(images.size(0)):
#             img = images[i]
#             label = labels[i].item()

#             pil_img = transforms.ToPILImage()(img.cpu())

#             if degradation == "noise":
#                 pil_img = add_gaussian_noise(pil_img)
#             elif degradation == "blur":
#                 pil_img = apply_blur(pil_img)
#             elif degradation == "low_light":
#                 pil_img = low_light(pil_img)

#             img_tensor = transform(pil_img).unsqueeze(0).to(device)

#             with torch.no_grad():
#                 logits = model(img_tensor)
#                 probs = F.softmax(logits, dim=1)

#             confidence = probs.max().item()
#             entropy = compute_entropy(probs).item()
#             pred = probs.argmax(dim=1).item()

#             results.append({
#                 "correct": int(pred == label),
#                 "confidence": confidence,
#                 "entropy": entropy
#             })

#     return pd.DataFrame(results)

In [15]:
# df_clean = run_inference(test_loader)
# df_noise = run_inference(test_loader, "noise")
# df_blur = run_inference(test_loader, "blur")
# df_low = run_inference(test_loader, "low_light")

100%|██████████| 157/157 [01:01<00:00,  2.56it/s]


In [16]:
# def summarize(df):
#     return {
#         "Accuracy": df["correct"].mean(),
#         "Avg Confidence": df["confidence"].mean(),
#         "Avg Entropy": df["entropy"].mean()
#     }

# summary = pd.DataFrame.from_dict({
#     "Clean": summarize(df_clean),
#     "Gaussian Noise": summarize(df_noise),
#     "Blur": summarize(df_blur),
#     "Low Light": summarize(df_low)
# }, orient="index")

# summary

,Accuracy,Avg Confidence,Avg Entropy
Clean,0.8048,0.802783,0.567405
Gaussian Noise,0.0934,0.527348,1.217914
Blur,0.6079,0.675338,0.906566
Low Light,0.5987,0.612446,1.109588


**For Each Image**

In [9]:
def run_inference(loader, degradation=None):
    results = []
    img_id = 0

    for images, labels in tqdm(loader):
        for i in range(images.size(0)):
            img = images[i]
            label = labels[i].item()

            pil_img = transforms.ToPILImage()(img.cpu())

            if degradation == "noise":
                pil_img = add_gaussian_noise(pil_img)
            elif degradation == "blur":
                pil_img = apply_blur(pil_img)
            elif degradation == "low_light":
                pil_img = low_light(pil_img)

            img_tensor = transform(pil_img).unsqueeze(0).to(device)

            with torch.no_grad():
                logits = model(img_tensor)
                probs = F.softmax(logits, dim=1)

            confidence = probs.max().item()
            entropy = compute_entropy(probs).item()
            pred = probs.argmax(dim=1).item()

            results.append({
                "image_id": img_id,
                "true_label": label,
                "pred_label": pred,
                "correct": int(pred == label),
                "confidence": confidence,
                "entropy": entropy,
                "degradation": degradation if degradation else "clean"
            })

            img_id += 1

    return pd.DataFrame(results)

In [10]:
df_clean = run_inference(test_loader)
df_noise = run_inference(test_loader, "noise")
df_blur  = run_inference(test_loader, "blur")
df_low   = run_inference(test_loader, "low_light")

100%|██████████| 157/157 [00:59<00:00,  2.65it/s]


In [15]:
!mkdir -p failure-aware-cv/results/csv

In [14]:
import os
print(os.getcwd())

/content/failure-aware-cv


In [16]:
!ls failure-aware-cv/results

csv


In [20]:
df_clean.to_csv("results/eval_image_clean.csv", index=False)
df_noise.to_csv("results/eval_image_noise.csv", index=False)
df_blur.to_csv("results/eval_image_blur.csv", index=False)
df_low.to_csv("results/eval_image_low_light.csv", index=False)

In [21]:
import pandas as pd

df_clean = pd.read_csv("results/eval_image_clean.csv")
df_noise = pd.read_csv("results/eval_image_noise.csv")
df_blur  = pd.read_csv("results/eval_image_blur.csv")
df_low   = pd.read_csv("results/eval_image_low_light.csv")

In [22]:
def summarize(df):
    return {
        "Accuracy": df["correct"].mean(),
        "Avg Confidence": df["confidence"].mean(),
        "Avg Entropy": df["entropy"].mean()
    }

In [23]:
summary = pd.DataFrame.from_dict({
    "Clean": summarize(df_clean),
    "Gaussian Noise": summarize(df_noise),
    "Blur": summarize(df_blur),
    "Low Light": summarize(df_low)
}, orient="index")

summary

,Accuracy,Avg Confidence,Avg Entropy
Clean,0.8048,0.802783,0.567405
Gaussian Noise,0.0919,0.526565,1.218909
Blur,0.6079,0.675338,0.906566
Low Light,0.5987,0.612446,1.109588


In [24]:
summary.to_csv("results/final_summary.csv")

In [ ]:
# !git status
# !git add notebooks/
# !git commit -m "Baseline vs degraded evaluation on GPU"
# !git push

fatal: not a git repository (or any of the parent directories): .git
fatal: not a git repository (or any of the parent directories): .git
fatal: not a git repository (or any of the parent directories): .git
fatal: not a git repository (or any of the parent directories): .git
